# Verify ArUco Package Integration

This notebook verifies the `snap_fit.aruco` and `snap_fit.puzzle.sheet_aruco` modules.


In [ ]:
from pathlib import Path
import sys

import cv2
from loguru import logger as lg
import matplotlib.pyplot as plt
import numpy as np

from snap_fit.aruco.aruco_board import ArucoBoardGenerator
from snap_fit.aruco.aruco_detector import ArucoDetector
from snap_fit.config.aruco.aruco_board_config import ArucoBoardConfig
from snap_fit.config.aruco.aruco_detector_config import ArucoDetectorConfig
from snap_fit.puzzle.sheet_aruco import SheetAruco

In [ ]:
# 1. Generate Board
board_config = ArucoBoardConfig(markers_x=5, markers_y=7)
generator = ArucoBoardGenerator(board_config)
board_image = generator.generate_image()

plt.figure(figsize=(5, 7))
plt.imshow(board_image, cmap="gray")
plt.title("Generated Board")
plt.axis("off")
plt.show()

In [ ]:
# 2. Create Distorted Image
rows, cols = board_image.shape
src_points = np.float32([[0, 0], [cols, 0], [0, rows], [cols, rows]])  # pyright: ignore[reportArgumentType]
squeeze_amount = cols * 0.2
dst_points = np.float32(
    [
        [squeeze_amount, rows * 0.1],
        [cols - squeeze_amount, rows * 0.1],
        [0, rows],
        [cols, rows],
    ]  # pyright: ignore[reportArgumentType]
)
M = cv2.getPerspectiveTransform(src_points, dst_points)  # pyright: ignore[reportArgumentType, reportCallIssue]
distorted_image = cv2.warpPerspective(  # pyright: ignore[reportCallIssue]
    board_image,
    M,
    (cols, rows),
    borderValue=255,  # pyright: ignore[reportArgumentType]
)

distorted_path = Path("distorted_test.png")
cv2.imwrite(str(distorted_path), distorted_image)

plt.figure(figsize=(5, 7))
plt.imshow(distorted_image, cmap="gray")
plt.title("Distorted Image")
plt.axis("off")
plt.show()

In [ ]:
# 3. Test ArucoDetector directly
detector_config = ArucoDetectorConfig()
detector = ArucoDetector(generator, detector_config)
corners, ids, _ = detector.detect_markers(distorted_image)
rectified = detector.correct_perspective(distorted_image, corners, ids)  # pyright: ignore[reportArgumentType]

if rectified is not None:
    plt.figure(figsize=(5, 7))
    plt.imshow(rectified, cmap="gray")
    plt.title("Rectified Image (Direct)")
    plt.axis("off")
    plt.show()
else:
    print("Rectification failed")

In [ ]:
# 4. Test SheetAruco
sheet_aruco = SheetAruco(detector, crop_margin=20)
sheet = sheet_aruco.load_sheet(distorted_path)

plt.figure(figsize=(5, 7))
plt.imshow(sheet.img_orig, cmap="gray")
plt.title("SheetAruco Loaded Image")
plt.axis("off")
plt.show()